# 모델 원  

- 컨텐츠 기반 추천 알고리즘
    - item count : 영화 14,672개
    - input text : 제목, 개봉일, 장르, 감독, 출연배우, 시놉시스가 결합된 텍스트
    - vectorizer : 단어빈도-역문서빈도 벡터화 > 사이킷런, TF-IDF vectorizer
    - similarity : Cosine Similarity > 사이킷런, pairwise.cosine_similarity
    - result     : 유사도 상위 N=10 개의 영화 추천.
    - representation  : 영화 포스터로 시각적 표현.

0. installation

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

1. DATA preparation

In [2]:
# 경로 지정.
dir_path = os.getcwd()
file_path = 'movie_db_0331.csv'
path = os.path.join(dir_path, file_path)

In [3]:
# 데이터 불러오기
temp_df= pd.read_csv(path)

In [4]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14672 entries, 0 to 14671
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   serial       14672 non-null  int64 
 1   title        14672 non-null  object
 2   release      14672 non-null  int64 
 3   just_rating  14012 non-null  object
 4   imdb_rating  14012 non-null  object
 5   runtime      14672 non-null  object
 6   synopsis     14672 non-null  object
 7   director     14130 non-null  object
 8   actors       14050 non-null  object
 9   genre        14479 non-null  object
 10  poster_link  14672 non-null  object
 11  netflix      14672 non-null  int64 
 12  disney       14672 non-null  int64 
 13  wavve        14672 non-null  int64 
 14  watcha       14672 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 1.7+ MB


In [8]:
print(str(len(temp_df))+" 개의 아이템")
temp_df.head(5)


14672 개의 아이템


,serial,title,release,just_rating,imdb_rating,runtime,synopsis,director,actors,genre,poster_link,netflix,disney,wavve,watcha
0,261279536,효자,2022,55%,55%,1시간57분,엄마가 돌아가시고 얼마 지나지 않아 닥친 태풍 소식에 5명의 형제들은 함께 산소를 ...,LeeHoon-guk,"KimRoi-ha,LeeCheol-min,JungKyung-ho,ParkHyo-jo...","공포,코미디",https://images.justwatch.com/poster/261279536/...,0,0,1,0
1,259066109,황무지의 괴물,2022,54%,4.6,1시간31분,19세기 외딴 황무지에 세상과 단절된 채 살아가는 가족이 있다. 이 집의 어린 아들...,NaN,"InmaCuesta,AsierFlores,RobertoÁlamo,AlejandraH...","공포,스릴러,드라마",https://images.justwatch.com/poster/259066109/...,1,0,0,0
2,261668749,홈 팀,2022,74%,6,1시간35분,자격 정지 처분을 받은 NFL 감독 숀 페이턴. 아들이 선수로 뛰는 형편없는 유소년...,"CharlesKinnane,DanielKinnane","KevinJames,TaylorLautner,RobSchneider,JackieSa...","코미디,스포츠",https://images.justwatch.com/poster/261668749/...,1,0,0,0
3,259326899,해적: 도깨비 깃발,2022,74%,6.1,2시간6분,자칭 고려 제일검인 의적단 두목 무치와 바다를 평정한 해적선의 주인 해랑. 한 배에...,KimJoung-hoon,"KangHa-neul,HanHyo-joo,LeeKwang-soo,KwonSang-w...","코미디,액션",https://images.justwatch.com/poster/259326899/...,1,0,0,0
4,258528334,해리 포터 20주년: 리턴 투 호그와트,2022,89%,8,1시간43분,"‘해리포터와 마법사의 돌' 20주년 기념 스페셜. 다니엘 래드클리프, 루퍼트 그린트...","EranCreevy,JoePearlman,CaseyPatterson,GiorgioT...","DanielRadcliffe,RupertGrint,EmmaWatson,BonnieW...","다큐멘터리,가족",https://images.justwatch.com/poster/258528334/...,0,0,1,0


In [10]:
# 필요컬럼 셀렉트
copy = temp_df.copy()
df = temp_df[['title','release','genre','director','actors','synopsis']]

In [13]:
# 결측값 확인 > 핵심 데이터를 시놉시스로 보아, 결측값 보정 없음.
df.isna().sum() 

title         0
release       0
genre       193
director    542
actors      622
synopsis      0
dtype: int64

In [16]:
# 제목이 같으나, 개봉연도가 다른 영화의 개수 확인
df.title.duplicated().sum()

283

In [34]:
# 문자열을 합쳐서 input text 로 사용하기 위해 모든 컬럼의 데이터 타입을 문자열로 변경.
df = df.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14672 entries, 0 to 14671
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     14672 non-null  object
 1   release   14672 non-null  object
 2   genre     14672 non-null  object
 3   director  14672 non-null  object
 4   actors    14672 non-null  object
 5   synopsis  14672 non-null  object
dtypes: object(6)
memory usage: 687.9+ KB


In [37]:
df[0:1].values.tolist()

[['효자',
  '2022',
  '공포,코미디',
  'LeeHoon-guk',
  'KimRoi-ha,LeeCheol-min,JungKyung-ho,ParkHyo-joon,JeonWoon-jong,NaMi-Hee,YunWoon-Kyung,MoonKyeong-min,AnMin-yeong,KimMin-sik,KimFeel',
  '엄마가 돌아가시고 얼마 지나지 않아 닥친 태풍 소식에 5명의 형제들은 함께 산소를 찾아간다  그런데, 이게 도대체 무슨 일? 부서진 관 사이로 엄마의 시신이 온데간데 사라졌다?  알 수 없는 기막힌 상황에 집으로 돌아오자 좀비로 변한 엄마가 이들을 기다리는데...']]

In [41]:
" ".join(df[0:1].values.tolist()[0])

'효자 2022 공포,코미디 LeeHoon-guk KimRoi-ha,LeeCheol-min,JungKyung-ho,ParkHyo-joon,JeonWoon-jong,NaMi-Hee,YunWoon-Kyung,MoonKyeong-min,AnMin-yeong,KimMin-sik,KimFeel 엄마가 돌아가시고 얼마 지나지 않아 닥친 태풍 소식에 5명의 형제들은 함께 산소를 찾아간다  그런데, 이게 도대체 무슨 일? 부서진 관 사이로 엄마의 시신이 온데간데 사라졌다?  알 수 없는 기막힌 상황에 집으로 돌아오자 좀비로 변한 엄마가 이들을 기다리는데...'

In [51]:
# 결합 텍스트를 리스트에 담아 df 마지막 열에 추가.
input_text = []
df.fillna(' ', inplace=True)
for i in range(len(df)):
    var1 = " ".join(df[i:i+1].values.tolist()[0])
    input_text.append(var1)
len(input_text)

14672

In [53]:
df['input_text'] = input_text
# df.head()

,title,release,genre,director,actors,synopsis,input_text
0,효자,2022,"공포,코미디",LeeHoon-guk,"KimRoi-ha,LeeCheol-min,JungKyung-ho,ParkHyo-jo...",엄마가 돌아가시고 얼마 지나지 않아 닥친 태풍 소식에 5명의 형제들은 함께 산소를 ...,"효자 2022 공포,코미디 LeeHoon-guk KimRoi-ha,LeeCheol-..."
1,황무지의 괴물,2022,"공포,스릴러,드라마",,"InmaCuesta,AsierFlores,RobertoÁlamo,AlejandraH...",19세기 외딴 황무지에 세상과 단절된 채 살아가는 가족이 있다. 이 집의 어린 아들...,"황무지의 괴물 2022 공포,스릴러,드라마 InmaCuesta,AsierFlor..."
2,홈 팀,2022,"코미디,스포츠","CharlesKinnane,DanielKinnane","KevinJames,TaylorLautner,RobSchneider,JackieSa...",자격 정지 처분을 받은 NFL 감독 숀 페이턴. 아들이 선수로 뛰는 형편없는 유소년...,"홈 팀 2022 코미디,스포츠 CharlesKinnane,DanielKinnane ..."
3,해적: 도깨비 깃발,2022,"코미디,액션",KimJoung-hoon,"KangHa-neul,HanHyo-joo,LeeKwang-soo,KwonSang-w...",자칭 고려 제일검인 의적단 두목 무치와 바다를 평정한 해적선의 주인 해랑. 한 배에...,"해적: 도깨비 깃발 2022 코미디,액션 KimJoung-hoon KangHa-ne..."
4,해리 포터 20주년: 리턴 투 호그와트,2022,"다큐멘터리,가족","EranCreevy,JoePearlman,CaseyPatterson,GiorgioT...","DanielRadcliffe,RupertGrint,EmmaWatson,BonnieW...","‘해리포터와 마법사의 돌' 20주년 기념 스페셜. 다니엘 래드클리프, 루퍼트 그린트...","해리 포터 20주년: 리턴 투 호그와트 2022 다큐멘터리,가족 EranCreevy..."
...,...,...,...,...,...,...,...
14667,Flowers and Trees,1932,애니메이션,BurtGillett,"ClarenceNash,EstherCampbell,MarionDarlington,P...",A jealous stump threatens two trees that are i...,Flowers and Trees 1932 애니메이션 BurtGillett Clare...
14668,스킨 게임,1931,드라마,AlfredHitchcock,"C.V.France,HelenHaye,JillEsmond,EdmundGwenn,Jo...",부자 가족 힐크리스트는 자신들의 땅에 공장을 짓기 위해 가난한 농부가 보낸 투자사기...,"스킨 게임 1931 드라마 AlfredHitchcock C.V.France,Hele..."
14669,리치 앤 스트레인지,1931,"드라마,로맨스,코미디,스릴러",AlfredHitchcock,"HenryKendall,JoanBarry,PercyMarmont,BettyAmann...",영화는 노동자인 프레드와 그의 부인 에밀리가 그들이 살아있는 동안 평생 뭐든지 할 ...,"리치 앤 스트레인지 1931 드라마,로맨스,코미디,스릴러 AlfredHitchcoc..."
14670,공공의 적,1931,"드라마,범죄",WilliamA.Wellman,"JamesCagney,JeanHarlow,EdwardWoods,JoanBlondel...","금주령 시대, 탐 파워스는 술집, 당구장 등을 어슬렁대며 퍼티 노우즈 같은 불량한 ...","공공의 적 1931 드라마,범죄 WilliamA.Wellman JamesCagney..."


2. feature vectorization

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=0.01, max_df=0.99)
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(input_text)

In [125]:
# 계산이 느리지 않아서 희소행렬의 문제 및 시간의 제약이 크게 없다.
tfidf_matrix[1].shape

(1, 343578)

3. recommendation system

In [126]:
# cos U 
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim.shape, type(cos_sim)

((14672, 14672), numpy.ndarray)

In [127]:
indices = pd.Series(df.index, index=df.title)

In [128]:
# 추천함수 상위 10개
def get_recommendation(title, cos_sim=cos_sim):
    index = indices[title]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    result = df.title.iloc[movie_indices]
    return result

In [137]:
get_recommendation('스파이더맨: 노 웨이 홈') # 

5427        스파이더맨: 홈커밍
7840          더 마인즈 아이
2726     스파이더맨: 파 프롬 홈
3999      어벤져스: 인피니티 워
12126          스파이더맨 2
11391          스파이더맨 3
8324      어메이징 스파이더맨 2
6937          닥터 스트레인지
5058          크레이지 특공대
1871             닥터 데스
Name: title, dtype: object

In [129]:
get_recommendation('반지의 제왕: 반지 원정대')

12500       반지의 제왕: 두 개의 탑
12340        반지의 제왕: 왕의 귀환
9432            호빗: 뜻밖의 여정
8069          호빗: 다섯 군대 전투
2787         성기사단: 어둠의 마법사
11843                   킹콩
10899               러블리 본즈
4350                 모털 엔진
283      잉글리시 북 클럽: 반지의 제왕
13184                프라이트너
Name: title, dtype: object

In [130]:
get_recommendation('부산행')

6649       서울역
3956        염력
10249      도가니
9129       사이비
9226     롤러코스터
8719        거인
5849       더 킹
3296       기생충
4393       마약왕
10527     부당거래
Name: title, dtype: object

In [131]:
get_recommendation('증기선 윌리') # 애니메이션

8887                터키
8723             갓즈 포켓
14647          외로운 유령들
9080     신데렐라: 트랩 오브 허
9134            사랑과 전쟁
10864        부다페스트 로큰롤
12919        미키의 크리스마스
1160         추억의 검정고무신
3886                윌리
14562         플루토와 소시지
Name: title, dtype: object

In [132]:
get_recommendation('삼국지 관운장: 청룡언월도') # keyword 관우

10141            삼국지: 명장 관우
1433                신해석 삼국지
246                   진삼국무쌍
4187            삼국지: 황건적의 난
11123            삼국지: 용의 부활
9284              대명겁: 천하대전
4440     로스트 바이킹: 전쟁의 신의 아들
1520           삼국지 : 무신 조자룡
191                       킨
3148                   도술혈전
Name: title, dtype: object

In [133]:
get_recommendation('다크 나이트')

11952                          배트맨 비긴즈
9235     레고 배트맨: 더 무비 - DC 수퍼히어로즈 유나이트
9798                        다크 나이트 라이즈
5756                         레고 배트맨 무비
13325                          배트맨 포에버
1072                    토네이도: 지구 재앙의 날
409                             아웃브레이크
6026                      감염: 지구 최후의 날
8648                              닌자터틀
6727                      배트맨: 더 킬링 조크
Name: title, dtype: object

In [134]:
get_recommendation('쇼생크 탈출')

12946          그린 마일
8408              설계
3533     해브 어 나이스 데이
12452         언디스퓨티드
11162          멋진 하루
1023            프리즈너
11431            미스트
5522             빠삐용
14216           사부출마
14383      돈을 갖고 튀어라
Name: title, dtype: object

4. 결과 도출 및 저장

In [ ]:
ab_list = df['title'][:]
len(ab_list), # ab_list

In [153]:
df_result = pd.DataFrame()    
for i in ab_list:
    try : 
        s = pd.DataFrame(get_recommendation(i))
        s.columns = [i]   # 컬럼네임 title => 영화제목으로 변경
        # s.rename(columns={'title':i})
        s.reset_index(drop=True, inplace=True) # 인덱스, 컨캣 시 에러를 띄우는 인덱스 정렬용.
        df_result = pd.concat([df_result, s], axis=1)  # 열 방향으로 컨캣
    except:
        # print(i)  # 키 중복으로 겟 레커멘드가. 영화를 특정하지 못함. 인덱스 반환으로 제목 확인. 중복된 제목임.
        pass
df_result.T

,0,1,2,3,4,5,6,7,8,9
효자,수상한 동거,괴물,손님,맨홀,내 생애 가장 아름다운 일주일,절대연필,바람아 안개를 걷어가다오,엄마의 레시피,Z 파이터스,지구를 지켜라!
황무지의 괴물,다른 길이 있다,쾌걸 조로,맨 인 더 다크 2,룸,네레우스: 바다의 괴물,하우스 오브 데몬,트롤 킹,사하라,셰터드,늑대의 아이들
홈 팀,휴비의 핼러윈,리디큘러스 6,쇼핑몰 캅 2,다 큰 녀석들,넌 실수였어,척 앤 래리,다 큰 녀석들 2,두 오버,2개의 심장,픽셀
해적: 도깨비 깃발,엘리아스 : 바다의 보물을 찾아라!,레오나르도 다 빈치,톰과 제리: 보물섬 대소동,꼬마잠수함 올리,탐정: 더 비기닝,군용탈보,캐리비안의 해적: 세상의 끝에서,페루 - 보물을 품은 그곳으로,소림사 2017: 7인의 고수,용문비갑
해리 포터 20주년: 리턴 투 호그와트,해리 포터와 불사조 기사단,해리 포터와 혼혈 왕자,해리 포터와 죽음의 성물 2,해리 포터와 아즈카반의 죄수,해리 포터와 비밀의 방,해리 포터와 불의 잔,해리 포터와 죽음의 성물 1,해리 포터와 마법사의 돌,신 테니스의 왕자 베스트 게임즈!! 복식경기 스페셜,신 테니스의 왕자- 베스트 게임즈!! 테즈카 vs 아토베
...,...,...,...,...,...,...,...,...,...,...
그랜드 호텔,미스터 마구,굿바이 마이 라이프,스미스씨 워싱톤에 가다,위대한 독재자,몬스터 호텔 3,밀리언 달러 호텔,그랜드 부다페스트 호텔,비욘드,트로트는 인생이다,아문센
Flowers and Trees,Fire in the Blood,Cobra Mafia,왕자가 된 소녀들,링링,물속의 도시,Lost in White,디지몬 세이버즈 더 무비 궁극파워! 버스트 모드 발동!!,Soul of Rock,Arena of the Street Fighter,The Tortoise and the Hare
스킨 게임,더 퍼지: 최초의 시작,듀플렉스,영 앤 이노센트,해리의 소동,허비: 돌아오다,39 계단,덤보,오 마이 파더!,더 디바이스,용재강호
리치 앤 스트레인지,럭키 원스,데스 룰렛,까시,리노의 도박사,크루즈 패밀리,비와 당신의 이야기,듀엣,저스트 프렌드,나의 연기 워크샵,서브웨이


In [168]:
df_result.T.to_csv('result_model_0404.csv', index=True, encoding='utf-8-sig')

#### comment :
    - n gram 방식을 취하면 상대적으로 감독 - 출연배우 / 배우 - 배우 조합에 따른 결과가 유사성이 높아질 수 있다. 

In [151]:
# 추가 사용 가능 함수들 정리

def get_index(title, df=df):
    index = df.index[(df['title'] == title)].to_list()
    return index 

def get_title(index, df=df):
    title = df['title'][(df.index == index)], df['synopsis'][index]
    return title

# 중복검출기 영화이름을 치면. 인덱스 반환
# print(df.index[(df['title'] == '연인')].to_list())

# get_index('부산행')
get_index('핵소 고지')

[12096, 13544]


[6202]

In [149]:
get_index('핵소 고지')

[6202]

In [150]:
def get_title(index, df=df):
    title = df['title'][(df.index == index)], df['synopsis'][index]
    return title

get_title(6202)

(6202    핵소 고지
 Name: title, dtype: object,
 '비폭력주의자인 도스는 전쟁으로부터 조국과 소중한 사람들을 지키기 위해 총을 들지 않아도 되는 의무병으로 육군에 자진 입대한다. 총을 들 수 없다는 이유로 필수 훈련 중 하나인 총기 훈련 마저 거부한 도스는 동료 병사들과 군 전체의 비난과 조롱을 받게 된다. 결국 군사재판까지 받게 되지만 끝까지 자신의 신념을 굽히지 않은 도스에게 군 상부는 오키나와 전투에 총기 없이 의무병으로 참전할 것을 허락하는데…')